In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import *

In [0]:
df = spark.read.load('/FileStore/tables/googleplaystore-3.csv', format='csv', sep=',', header='true', escape='"', inferschema='true')

In [0]:
df.count()

Out[5]: 10841

In [0]:
df.show(1)

+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|      Genres|   Last Updated|Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M| 10,000+|Free|    0|      Everyone|Art & Design|January 7, 2018|      1.0.0|4.0.3 and up|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
only showing top 1 row



In [0]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [0]:
#dropping a few columns that may not be so important.
df = df.drop("size", "Content Rating", "Last Updated", "Android Ver", "Current Ver")

In [0]:
df.show(1)

+--------------------+--------------+------+-------+--------+----+-----+------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|      Genres|
+--------------------+--------------+------+-------+--------+----+-----+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 10,000+|Free|    0|Art & Design|
+--------------------+--------------+------+-------+--------+----+-----+------------+
only showing top 1 row



In [0]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)



In [0]:
#converting some column data types

#converts the review column from string to integer type
df = df.withColumn("Reviews", col("Reviews").cast(IntegerType()))

In [0]:
from pyspark.sql.functions import regexp_replace, col
#removing + symbol from symbol, $ symbol from price and converting both to integer data types

df = df.withColumn("Installs", regexp_replace(col("Installs"), "[^0-9]", ""))\
    .withColumn("Installs", col("Installs").cast(IntegerType()))


In [0]:
df = df.withColumn("Price", regexp_replace(col("Price"), "[$]", ""))\
    .withColumn("Price", col("Price").cast(IntegerType()))

In [0]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Genres: string (nullable = true)



In [0]:
df.show(5)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|    0|Art & Design;Crea...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 5 rows



In [0]:
#convert the data frame into a view

df.createOrReplaceTempView("apps")

In [0]:
# %sql select * from apps

In [0]:
%sql select App, sum(Reviews) from apps
group by App
order by 2 desc
LIMIT 10

App,sum(Reviews)
Instagram,266241989
WhatsApp Messenger,207348304
Clash of Clans,179558781
Messenger – Text and Video Chat for Free,169932272
Subway Surfers,166331958
Candy Crush Saga,156993136
Facebook,156286514
8 Ball Pool,99386198
Clash Royale,92530298
Snapchat,68045010


In [0]:
%sql select App, sum(Installs) From apps
group by 1
order by 2 desc
LIMIT 10

App,sum(Installs)
Subway Surfers,6000000000
Instagram,4000000000
Hangouts,4000000000
Google Drive,4000000000
Google News,4000000000
Google Photos,4000000000
Candy Crush Saga,3500000000
WhatsApp Messenger,3000000000
Messenger – Text and Video Chat for Free,3000000000
Google Chrome: Fast & Secure,3000000000


In [0]:
%sql select App, sum(Price) from apps
where Type='Paid'
group by 1
order by 2 desc
LIMIT 10

App,sum(Price)
I'm Rich - Trump Edition,400
most expensive app (H),399
I am rich(premium),399
I Am Rich Premium,399
I am Rich Plus,399
I'm Rich/Eu sou Rico/أنا غني/我很有錢,399
I Am Rich Pro,399
I AM RICH PRO PLUS,399
I am Rich,399
I am Rich!,399


Databricks visualization. Run in Databricks to view.